In [35]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import random
import seaborn as sns
from sklearn import tree
import sklearn.ensemble as ske
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [36]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

 # linear algebra
 # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [37]:
#loading ds
train=pd.read_csv('/kaggle/input/titanic/train.csv')
test=pd.read_csv('/kaggle/input/titanic/test.csv')

In [38]:
#check ds
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [39]:
#check ds
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [40]:

from sklearn.impute import KNNImputer
imputer=KNNImputer(n_neighbors=20)

train.Age=imputer.fit_transform(train.Age.values.reshape(-1,1))
test.Age=imputer.fit_transform(test.Age.values.reshape(-1,1))

train.Age=train.Age.astype(int)
test.Age=test.Age.astype(int)

In [41]:
train.Fare=imputer.fit_transform(train.Fare.values.reshape(-1,1))
test.Fare=imputer.fit_transform(test.Fare.values.reshape(-1,1))

In [42]:
train['Cabin']=train['Cabin'].fillna(train['Cabin'].mode()[0])
test['Cabin']=test['Cabin'].fillna(test['Cabin'].mode()[0])

train['Embarked']=train['Embarked'].fillna(train['Embarked'].mode()[0])
test['Embarked']=test['Embarked'].fillna(test['Embarked'].mode()[0])

In [43]:
train['Title']=train['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
test['Title']=test['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())

In [44]:
title=train.groupby('Title')['Survived'].sum().reset_index()
title

,Title,Survived
0,Capt,0
1,Col,1
2,Don,0
3,Dr,3
4,Jonkheer,0
5,Lady,1
6,Major,1
7,Master,23
8,Miss,127
9,Mlle,2


In [45]:
#mapping titles
title1={'Capt':1,'Col':2,'Don':3,'Dr':4,'Jonkheer':5,'Lady':6,'Major':7,'Master':8,'Miss':9,
        'Mlle':10,'Mme':11,'Mr':12,'Mrs':13,'Ms':14,'Rev':15,'Sir':16,'the Countess':17,'Dona':18}
train.Title=train.Title.map(title1)
test.Title=test.Title.map(title1)

In [46]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

train.Sex=le.fit_transform(train.Sex)
test.Sex=le.fit_transform(test.Sex)

In [47]:
bins=[-1.,18.,35.,64.,65.+np.inf]
names=['child','young adult','middle aged','pensioner']

train['Age_Range']=pd.cut(train['Age'],bins,labels=names)
test['Age_Range']=pd.cut(test['Age'],bins,labels=names)



In [48]:
train.Age_Range=le.fit_transform(train.Age_Range)
test.Age_Range=le.fit_transform(test.Age_Range)

In [49]:
train['Family']=train.SibSp + train.Parch
test['Family']=test.SibSp + test.Parch
train.Family.sum(),test.Family.sum()

(806, 351)

In [50]:
bins0=[-1.,100.,200.,300.,400.,500.+ np.inf]
names0=['0-99','100-199','200-299','300-399','400+']

train['Fare_Range']=pd.cut(train['Fare'],bins0,labels=names0)
test['Fare_Range']=pd.cut(test['Fare'],bins0,labels=names0) 

In [51]:
train.Fare_Range=le.fit_transform(train.Fare_Range)
test.Fare_Range=le.fit_transform(test.Fare_Range)

In [52]:
train.Embarked=le.fit_transform(train.Embarked)
test.Embarked=le.fit_transform(test.Embarked)

In [53]:
y=train["Survived"]
features=["Pclass","Sex","Family","Embarked","Title","Age_Range","Fare_Range"]
X=pd.get_dummies(train[features])
X_test=pd.get_dummies(test[features])

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_validation, Y_train, Y_validation=train_test_split(X,y,test_size=0.20, random_state=1)

In [55]:
#random foreset classifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

selection=LogisticRegression(penalty='l1', solver='liblinear')
selection.fit(X_train,Y_train)
penalty=['l1','l2']
C=np.logspace(0,4,10)
hyperparameters=dict(C=C,penalty=penalty)

clf=GridSearchCV(selection, hyperparameters, cv=5, verbose=0)

best_model=clf.fit(X_train, Y_train)
print(clf.score(X_train, Y_train))


    

0.8061797752808989


In [56]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

name='LR'
kfold = StratifiedKFold(n_splits=10,random_state=1,shuffle=True)
cv_results = cross_val_score(best_model, X_train, Y_train, cv=kfold, scoring='accuracy')
print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

LR: 0.794933 (0.042378)


In [57]:
y_pred=clf.predict(X_validation)

In [58]:
df=pd.DataFrame({'Actual': Y_validation,'Predicted':y_pred})
df

,Actual,Predicted
862,1,1
223,0,0
84,1,1
680,0,1
535,1,1
...,...,...
796,1,1
815,0,0
629,0,0
421,0,0


In [59]:
#decission tree
clf_dt = tree. DecisionTreeClassifier(max_depth=10)

In [60]:
clf_dt.fit (X_train, Y_train)

DecisionTreeClassifier(max_depth=10)

In [61]:
#gaussian classification
gnb = GaussianNB()
gnb.fit(X_train, Y_train)
y_pred = gnb.predict(X_train)
acc = accuracy_score(Y_train, y_pred)
acc

0.7949438202247191

In [62]:
predictions = clf.predict(X_test)

In [63]:
output= pd.DataFrame({'PassengerId': test.PassengerId,'Survived': predictions})
output.to_csv('assignment.csv',index=False)
print("submission  saved")

submission  saved


In [64]:
assignment=pd.read_csv("assignment.csv")
assignment

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
